In [3]:
# Wormy (a Nibbles clone)
# By Al Sweigart al@inventwithpython.com
# http://inventwithpython.com/pygame
# Released under a "Simplified BSD" license

import random, pygame, sys, time #import에 time 추가
from pygame.locals import *

FPS = 15
WINDOWWIDTH = 640
WINDOWHEIGHT = 480
CELLSIZE = 20
assert WINDOWWIDTH % CELLSIZE == 0, "Window width must be a multiple of cell size."
assert WINDOWHEIGHT % CELLSIZE == 0, "Window height must be a multiple of cell size."
CELLWIDTH = int(WINDOWWIDTH / CELLSIZE)
CELLHEIGHT = int(WINDOWHEIGHT / CELLSIZE)

#             R    G    B
WHITE     = (255, 255, 255)
BLACK     = (  0,   0,   0)
RED       = (255,   0,   0)
GREEN     = (  0, 255,   0)
DARKGREEN = (  0, 155,   0)
DARKGRAY  = ( 40,  40,  40)
BGCOLOR = BLACK
YELLOW    = (255, 255,   0) # 시작 화면의 폰트 노란 색깔
LIGHTYELLOW=(255, 255, 150) #시작 화면의 폰트 밝은 노란 색깔

UP = 'up'
DOWN = 'down'
LEFT = 'left'
RIGHT = 'right'

HEAD = 0 # index 0을 지렁이 헤드로 지정

TEXTCOLOR = WHITE #text 색


def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT
    #pygame 시작
    pygame.init()
    #게임 프레임속도 제어
    FPSCLOCK = pygame.time.Clock()
    #게임 화면 설정
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    #기본 폰트 설정
    BASICFONT = pygame.font.Font('D2Coding.ttc', 18)
    # 상단바 이름 설정
    pygame.display.set_caption('2019094266_Kimminsu')

    #시작화면을 불어오는 showStartScreen 함수 호출
    showStartScreen()
    #게임이 완전히 종료되기 전까지 랜덤 음악 실행, runGame, showGameOverScreen 함수 호출
    while True:
        #랜덤 음악 실행
        random_music = random.randint(0,2)
        if random_music == 0:
            pygame.mixer.music.load('Our_Lives_Past.mp3')
        elif random_music ==1 :
            pygame.mixer.music.load('Platform_9.mp3')
        else:
            pygame.mixer.music.load('Hover.mp3')
        pygame.mixer.music.play(-1, 0.0)
        runGame()
        showGameOverScreen()
        

def runGame():
    global start_ticks,elapsed_time #전역변수로 start_ticks,elapsed_time 지정
    # 지렁이 랜덤 생성 위치 설정
    startx = random.randint(5, CELLWIDTH - 6)
    starty = random.randint(5, CELLHEIGHT - 6)
    wormCoords = [{'x': startx,     'y': starty},
                  {'x': startx - 1, 'y': starty},
                  {'x': startx - 2, 'y': starty}]
    direction = RIGHT

    # 사과 랜덤 생성 위치 설정
    apple = getRandomLocation()
    
    #게임 시작시간 정보
    start_ticks = pygame.time.get_ticks()

    while True: # 메인 게임 루프
        for event in pygame.event.get(): # 이벤트 처리 루프
            if event.type == QUIT:  # esc키를 누르면 게임 종료
                terminate()
            elif event.type == KEYUP:
                #게임 pause 기능(p키를 누르면 pause)
                if (event.key == K_p):
                    DISPLAYSURF.fill(BGCOLOR)
                    pygame.mixer.music.stop() #음악 일시 정지
                    showTextScreen('Paused') # 화면에 Paused 보여주기
                    pygame.mixer.music.play(-1, 0.0) #음악 다시 실행
            #방향키를 통해 방향을 바꿔줌
            elif event.type == KEYDOWN:
                if (event.key == K_LEFT or event.key == K_a) and direction != RIGHT:
                    direction = LEFT
                elif (event.key == K_RIGHT or event.key == K_d) and direction != LEFT:
                    direction = RIGHT
                elif (event.key == K_UP or event.key == K_w) and direction != DOWN:
                    direction = UP
                elif (event.key == K_DOWN or event.key == K_s) and direction != UP:
                    direction = DOWN
                elif event.key == K_ESCAPE:
                    terminate()

        # 지렁이가 화면 끝에 닿으면 게임 오버
        if wormCoords[HEAD]['x'] == -1 or wormCoords[HEAD]['x'] == CELLWIDTH or wormCoords[HEAD]['y'] == -1 or wormCoords[HEAD]['y'] == CELLHEIGHT:
            return # 게임 오버
        # 지렁이의 머리가 자신의 몸에 닿으면 게임 오버
        for wormBody in wormCoords[1:]:
            if wormBody['x'] == wormCoords[HEAD]['x'] and wormBody['y'] == wormCoords[HEAD]['y']:
                return # 게임 오버

        # 지렁이의 머리가 사과를 먹으면
        if wormCoords[HEAD]['x'] == apple['x'] and wormCoords[HEAD]['y'] == apple['y']:
            # 사과 위치 랜덤 생성
            apple = getRandomLocation() 
        #????
        else:
            del wormCoords[-1] 

        # 이동 방향에 따라 머리 생성위치를 설정
        if direction == UP:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y'] - 1}
        elif direction == DOWN:
            newHead = {'x': wormCoords[HEAD]['x'], 'y': wormCoords[HEAD]['y'] + 1}
        elif direction == LEFT:
            newHead = {'x': wormCoords[HEAD]['x'] - 1, 'y': wormCoords[HEAD]['y']}
        elif direction == RIGHT:
            newHead = {'x': wormCoords[HEAD]['x'] + 1, 'y': wormCoords[HEAD]['y']}
        
        #wormCoords 앞에 새로운 머리 
        wormCoords.insert(0, newHead)
        #화면을 BGCLOLOR로 채움
        DISPLAYSURF.fill(BGCOLOR)
        #격자 그림
        drawGrid()
        #wormCoords를 화면에 나타냄
        drawWorm(wormCoords)
        #apple을 화면에 나타냄
        drawApple(apple)
        #경과 시간 계산, 경과 시간을 1000으로 나눠 초 단위로 표시
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        #wormCoords 길이에 대해 점수와 플레이타임을 표시
        drawStatus(len(wormCoords) - 3,len(wormCoords) - 3)
        #화면 전환
        pygame.display.update()
        #지정 fps로 실행
        FPSCLOCK.tick(FPS)

def drawPressKeyMsg():
    #화면에 press a key to play 출력
    pressKeySurf = BASICFONT.render('Press a key to play.', True, DARKGRAY)
    pressKeyRect = pressKeySurf.get_rect()
    #글자 위치 설정
    pressKeyRect.topleft = (WINDOWWIDTH - 200, WINDOWHEIGHT - 30)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)


def checkForKeyPress():
    #quit가 하나라도 있으면 게임 종료
    if len(pygame.event.get(QUIT)) > 0:
        terminate()

    keyUpEvents = pygame.event.get(KEYUP)
    #keyupevents가 없으면 none 리턴
    if len(keyUpEvents) == 0:
        return None
    #첫 키가 esc면 게임 종료
    if keyUpEvents[0].key == K_ESCAPE:
        terminate()
    #첫 키 리턴
    return keyUpEvents[0].key


def showStartScreen():
    titleFont = pygame.font.Font('D2Coding.ttc', 100) #폰트 설정
    #시작화면 자막 설정
    titleSurf1 = titleFont.render('OSW Game', True, BLACK, LIGHTYELLOW)
    titleSurf2 = titleFont.render('OSW Game', True, YELLOW)

    degrees1 = 0
    degrees2 = 0
    while True:
        DISPLAYSURF.fill(BGCOLOR)
        rotatedSurf1 = pygame.transform.rotate(titleSurf1, degrees1)
        rotatedRect1 = rotatedSurf1.get_rect()
        rotatedRect1.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf1, rotatedRect1)

        rotatedSurf2 = pygame.transform.rotate(titleSurf2, degrees2)
        rotatedRect2 = rotatedSurf2.get_rect()
        rotatedRect2.center = (WINDOWWIDTH / 2, WINDOWHEIGHT / 2)
        DISPLAYSURF.blit(rotatedSurf2, rotatedRect2)

        drawPressKeyMsg()
        #ecs가 아닌 키를 누르면 while 루프 탈출
        if checkForKeyPress():
            pygame.event.get()
            return
        pygame.display.update()
        FPSCLOCK.tick(FPS)
        #프레임 당 각도 설정
        degrees1 += 3 
        degrees2 += 7


def terminate():
    pygame.quit()
    sys.exit()


def getRandomLocation():
    return {'x': random.randint(0, CELLWIDTH - 1), 'y': random.randint(0, CELLHEIGHT - 1)}


def showGameOverScreen():
    #game over 자막 화면에 출력
    gameOverFont = pygame.font.Font('D2Coding.ttc', 150)
    gameSurf = gameOverFont.render('Game', True, WHITE)
    overSurf = gameOverFont.render('Over', True, WHITE)
    gameRect = gameSurf.get_rect()
    overRect = overSurf.get_rect()
    gameRect.midtop = (WINDOWWIDTH / 2, 10)
    overRect.midtop = (WINDOWWIDTH / 2, gameRect.height + 10 + 25)
    DISPLAYSURF.blit(gameSurf, gameRect)
    DISPLAYSURF.blit(overSurf, overRect)
    #press a key to play 자막 화면에 출력
    drawPressKeyMsg()
    pygame.display.update()
    pygame.time.wait(500)
    checkForKeyPress()

    while True:
        #esc가 아닌 다른 키 입력시 while루프 탈출
        if checkForKeyPress():
            pygame.event.get()
            return

def drawStatus(score,playtime): #drawScore을 drawStatus로 변경 후 playtime 추가
    #score을 화면 상단에 출력
    scoreSurf = BASICFONT.render('Score: %s' % (score), True, WHITE)
    scoreRect = scoreSurf.get_rect()
    scoreRect.topleft = (WINDOWWIDTH - 150, 10)
    DISPLAYSURF.blit(scoreSurf, scoreRect)
    
    #playtime을 화면 상단 score 아래 출력
    playtimeSurf = BASICFONT.render('Playtime: %s sec' % (str(int(elapsed_time))), True, WHITE)
    playtimeRect = playtimeSurf.get_rect()
    playtimeRect.topleft = (WINDOWWIDTH - 150, 30)
    DISPLAYSURF.blit(playtimeSurf, playtimeRect)
    
    pygame.display.update()


def drawWorm(wormCoords):
    for coord in wormCoords:
        x = coord['x'] * CELLSIZE
        y = coord['y'] * CELLSIZE
        wormSegmentRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
        pygame.draw.rect(DISPLAYSURF, DARKGREEN, wormSegmentRect)
        wormInnerSegmentRect = pygame.Rect(x + 4, y + 4, CELLSIZE - 8, CELLSIZE - 8)
        pygame.draw.rect(DISPLAYSURF, GREEN, wormInnerSegmentRect)


def drawApple(coord):
    x = coord['x'] * CELLSIZE
    y = coord['y'] * CELLSIZE
    appleRect = pygame.Rect(x, y, CELLSIZE, CELLSIZE)
    pygame.draw.rect(DISPLAYSURF, RED, appleRect)


def drawGrid():
    #수직선 화면에 생성
    for x in range(0, WINDOWWIDTH, CELLSIZE): # draw vertical lines
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (x, 0), (x, WINDOWHEIGHT))
    #평행성 화면에 생성
    for y in range(0, WINDOWHEIGHT, CELLSIZE): # draw horizontal lines
        pygame.draw.line(DISPLAYSURF, DARKGRAY, (0, y), (WINDOWWIDTH, y))
    
def makeTextObjs(text, font, color):
    surf = font.render(text, True, color)
    return surf, surf.get_rect()
    
def showTextScreen(text): 

    titleSurf, titleRect = makeTextObjs(text, BASICFONT,TEXTCOLOR)
    titleRect.center = (int(WINDOWWIDTH / 2) - 3, int(WINDOWHEIGHT / 2) - 3)
    DISPLAYSURF.blit(titleSurf, titleRect)

    pressKeySurf, pressKeyRect = makeTextObjs('Press a key to play.', BASICFONT, TEXTCOLOR)
    pressKeyRect.center = (int(WINDOWWIDTH / 2), int(WINDOWHEIGHT / 2) + 100)
    DISPLAYSURF.blit(pressKeySurf, pressKeyRect)

    while checkForKeyPress() == None:
        pygame.display.update()
        FPSCLOCK.tick()

        
if __name__ == '__main__':
    main()

SystemExit: 